# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../../..")))

## 1. Read Informal Proves
Read the LaTeX file. This file contains the informal proofs of the theorems in the book.

In [2]:
from IPython.display import display, Math, Latex
import re
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof_latex = read_proof("../../data/proofs/english/divisible_by_eight/proof1.tex")

# Find the start and end positions
start = proof_latex.find(r"\begin{document}") + len(r"\begin{document}")
end = proof_latex.find(r"\end{document}")

# Extract the content between \begin{document} and \end{document}
informal_proof = proof_latex[start:end].strip()

# Replace any \section{...} with ## ...
informal_proof = re.sub(r"\\section\{([^}]+)\}", r"## \1", informal_proof)
informal_proof = re.sub(r"\\subsection\{([^}]+)\}", r"### \1", informal_proof)
informal_proof = re.sub(r"\\title\{([^}]+)\}", r"# \1", informal_proof)
informal_proof = re.sub(r"\\maketitle", "", informal_proof)

# Replace \begin{itemize} and \end{itemize} with Markdown-style lists
informal_proof = re.sub(r"\\begin{itemize}", "", informal_proof)
informal_proof = re.sub(r"\\end{itemize}", "", informal_proof)
informal_proof = re.sub(r"\\item\s+\*\*([^:]+):\*\*", r"- **\1:**", informal_proof)

# Display the LaTeX content
display(Latex(informal_proof))

<IPython.core.display.Latex object>

## 2. Extract Triplet proofs from Informal Proofs

In [3]:
from src.phase1.extract_triplets import extract_triplets

# Extract triplets
triplet = extract_triplets(informal_proof)
triplet

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Triplet(entities=[Entity(id='1', name='a', label='Odd Integer', type='Variable'), Entity(id='2', name='b', label='Odd Integer', type='Variable'), Entity(id='3', name='k', label='Integer', type='Variable'), Entity(id='4', name='m', label='Integer', type='Variable'), Entity(id='5', name='n', label='Integer', type='Variable'), Entity(id='6', name='8', label='Integer', type='Constant'), Entity(id='7', name='a^2 - b^2', label='Difference of Squares', type='Expression'), Entity(id='8', name='4(k - m)(k + m + 1)', label='Expression for Difference of Squares', type='Expression'), Entity(id='9', name='8n(k + m + 1)', label='Expression for Case 1', type='Expression'), Entity(id='10', name='8n(k - m)', label='Expression for Case 2', type='Expression')], relations=[Relation(source='1', target='3', type='represents', name='Representation of a'), Relation(source='2', target='4', type='represents', name='Representation of b'), Relation(source='3', target='5', type='is an element of', name='k and m ar

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## 3. Store Triplets into Neo4J DB

In [4]:
from src.utils.neo4j_utils import Neo4JUtils

# Initialize Neo4JUtils
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))

# Clean the database (delete all nodes and relationships)
neo4j.clean_database()

# Add nodes and relationships with step tracking
for entity in triplet.entities:
    neo4j.create_node(entity)  # Uses the current step (default is 0)
for relation in triplet.relations:
    neo4j.create_relation(relation)  # Uses the current step (default is 0)

# Increment the step counter for the next set of changes
neo4j.increment_step()

# Clean the database (delete nodes and relationships with step > 1)
neo4j.clean_database(step=1)

# Add or modify nodes and relationships in the next step
# Example:
# neo4j.create_node(new_entity)  # This will use the updated step counter (1)
# neo4j.create_relation(new_relation)  # This will use the updated step counter (1)

# Close the connection
neo4j.close()